In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import sys
from datetime import datetime
from guppy import hpy

# Create an instance of heapy
hp = hpy()

# Define a function to open datasets and concatenate them
def open_and_concatenate(year, variable, months, way, level=0):
    datasets = [xr.open_dataset(f'{way}{variable}/ERA5_{year}-{month}_{variable}.nc') for month in months]
    if variable == 'geopotential' and level != 0:
        datasets = [dataset.sel(level=level) for dataset in datasets]
    return xr.concat(datasets, dim='time')

# Define a function to calculate statistics
def calculate_statistics(data_array):
    return {
        'mean': np.mean(data_array),
        'min': np.min(data_array),
        'max': np.max(data_array),
        'std': np.std(data_array),
    }

# Function to log processing details
def log_processing(variable, year, level, storm_number):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_message = f'Processed variable: {variable}, Year: {year}, Level: {level}, Timestamp: {timestamp}, Storm number:{storm_number}'
    with open(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/datasets/processing_log.txt', 'a') as log_file:
        log_file.write(log_message + '\n')

# Main function to process data
def process_data(variable, year, level=0):
    year = int(year)
    year_next = year + 1
    month_act = [10, 11, 12]
    month_next = [1, 2, 3]
    if variable == 'geopotential':
        way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5_hourly_PL/'
    else:
        way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'

    # Open and concatenate datasets
    if year == 1990:
        dataset_act = open_and_concatenate(str(year), variable, month_next, way, level)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')
        dataset = dataset.chunk({'time': 10})
    elif year == 2021:
        dataset = open_and_concatenate(str(year), variable, month_next, way, level)
    else:
        dataset_act = open_and_concatenate(str(year), variable, month_act, way, level)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')
        dataset = dataset.chunk({'time': 10})

    # Determine the specific variable to extract
    specific_var = next(var for var in dataset.variables if var not in ['longitude', 'latitude', 'time', 'level'])

    # Import all tracks and convert dates
    dates = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/storms_start_end.csv', parse_dates=['start_date', 'end_date'])
    dates['year'] = dates['start_date'].dt.year

    # Find the indices for storms within the specified timeframe
    if year == 1990:
        index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
        index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]
    elif year == 2021:
        index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
        index_end_march = dates[(dates['end_date'].dt.year == 2021)].index[0]
    else:
        index_start_october = dates[((dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)) | ((dates['start_date'].dt.year == year_next) & (dates['start_date'].dt.month >= 1))].index[0]
        index_end_march_first = dates[((dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next))].index
        if len(index_end_march_first) > 0:
            index_end_march = index_end_march_first[-1]
        else:
            index_end_march = dates[((dates['end_date'].dt.year == year) & (dates['end_date'].dt.month <= 12))].index[-1]

    # Process each storm
    for i in range(index_start_october, index_end_march + 1):
        track = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/tc_irad_tracks/tc_1_hour/tc_irad_{i+1}_interp.txt')
        start_date = dates.at[i, 'start_date']
        end_date = dates.at[i, 'end_date']
        storm_data = dataset[specific_var].sel(time=slice(start_date, end_date))

        # Initialize lists to store statistics
        stats = {'mean': [], 'min': [], 'max': [], 'std': []}

        # Process each time step
        for t_index, time_step in enumerate(storm_data.time):
            #data_slice = storm_data.sel(time=time_step).values

            # Extract coordinates for the current time step
            lon_e_temp, lon_w_temp, lat_s_temp, lat_n_temp = track.iloc[t_index]
            lon_test = np.asanyarray(storm_data.longitude[:])
            lat_test = np.asanyarray(storm_data.latitude[:])

            closest_lon_w = np.abs(lon_test - lon_w_temp).argmin()
            closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
            closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
            closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

            closest_lon_w_coor = lon_test[closest_lon_w]
            closest_lon_e_coor = lon_test[closest_lon_e]
            closest_lat_s_coor = lat_test[closest_lat_s]
            closest_lat_n_coor = lat_test[closest_lat_n]

            # Use .roll to handle the 0°/360° boundary
            if closest_lon_w_coor < 100 and closest_lon_e_coor > 100:
                roll_shift = {'longitude': int(round(closest_lon_w_coor, 0)), 'longitude': int(round(closest_lon_e_coor, 0))}
                storm_data_rolled = storm_data.roll(roll_shift, roll_coords=True)
            else:
                storm_data_rolled = storm_data

            # Slice the dataset based on the rolled longitudes and latitudes
            temp_ds_time = storm_data_rolled.isel(time=t_index)#[specific_var].isel(time=t_index)
            temp_ds = temp_ds_time.sel(latitude=slice(closest_lat_n_coor, closest_lat_s_coor),
                                       longitude=slice(closest_lon_e_coor, closest_lon_w_coor)).values

            # Calculate statistics for the sliced data
            step_stats = calculate_statistics(temp_ds)
            for key in stats:
                stats[key].append(step_stats[key])

        # Save statistics to CSV files
        for key in stats:
            pd.DataFrame(stats[key]).to_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/datasets/{variable}/storm_{i+1}/{key}_{i+1}_{level}.csv')

    # Log the processing details
    log_processing(variable, year, level, i+1)

if __name__ == '__main__':
    variable = '2m_temperature'
    year = 2006
    level = 0
    process_data(variable, year, level)

# Obtain a snapshot of memory usage
h = hp.heap()

# Print memory usage information
print(h)

KeyboardInterrupt: 

In [ ]:
year = int(year)
year_next = year + 1
month_act = [10, 11, 12]
month_next = [1, 2, 3]
if variable == 'geopotential':
    way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5_hourly_PL/'
else:
    way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'

# Open and concatenate datasets
if year == 1990:
    dataset_act = open_and_concatenate(str(year), variable, month_next, way, level)
    dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
    dataset = xr.concat([dataset_act, dataset_next], dim='time')
    dataset = dataset.chunk({'time': 10})
elif year == 2021:
    dataset = open_and_concatenate(str(year), variable, month_next, way, level)
else:
    dataset_act = open_and_concatenate(str(year), variable, month_act, way, level)
    dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
    dataset = xr.concat([dataset_act, dataset_next], dim='time')
    dataset = dataset.chunk({'time': 10})

# Determine the specific variable to extract
specific_var = next(var for var in dataset.variables if var not in ['longitude', 'latitude', 'time', 'level'])

# Import all tracks and convert dates
dates = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/storms_start_end.csv', parse_dates=['start_date', 'end_date'])
dates['year'] = dates['start_date'].dt.year

# Find the indices for storms within the specified timeframe
if year == 1990:
    index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]
elif year == 2021:
    index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.year == 2021)].index[0]
else:
    index_start_october = dates[((dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)) | ((dates['start_date'].dt.year == year_next) & (dates['start_date'].dt.month >= 1))].index[0]
    index_end_march_first = dates[((dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next))].index
    if len(index_end_march_first) > 0:
        index_end_march = index_end_march_first[-1]
    else:
        index_end_march = dates[((dates['end_date'].dt.year == year) & (dates['end_date'].dt.month <= 12))].index[-1]

# Process each storm
for i in range(index_start_october, index_end_march+1):
    track = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/tc_irad_tracks/tc_1_hour/tc_irad_{i+1}_interp.txt')
    start_date = dates.at[i, 'start_date']
    end_date = dates.at[i, 'end_date']
    storm_data = dataset[specific_var].sel(time=slice(start_date, end_date))

    # Initialize lists to store statistics
    stats = {'mean': [], 'min': [], 'max': [], 'std': []}

    # Process each time step
    for t_index, time_step in enumerate(storm_data.time):
        #data_slice = storm_data.sel(time=time_step).values

        # Extract coordinates for the current time step
        lon_e_temp, lon_w_temp, lat_s_temp, lat_n_temp = track.iloc[t_index]
        lon_test = np.asanyarray(storm_data.longitude[:])
        lat_test = np.asanyarray(storm_data.latitude[:])

        closest_lon_w = np.abs(lon_test - lon_w_temp).argmin()
        closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
        closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
        closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

        closest_lon_w_coor = lon_test[closest_lon_w]
        closest_lon_e_coor = lon_test[closest_lon_e]
        closest_lat_s_coor = lat_test[closest_lat_s]
        closest_lat_n_coor = lat_test[closest_lat_n]

        # Use .roll to handle the 0°/360° boundary
        if closest_lon_w_coor < 100 and closest_lon_e_coor > 100:
            roll_shift = {'longitude': int(round(closest_lon_w_coor, 0)), 'longitude': int(round(closest_lon_e_coor, 0))}
            storm_data_rolled = storm_data.roll(roll_shift, roll_coords=True)
        else:
            storm_data_rolled = storm_data

        # Slice the dataset based on the rolled longitudes and latitudes
        temp_ds_time = storm_data_rolled.isel(time=t_index)#[specific_var].isel(time=t_index)
        temp_ds = temp_ds_time.sel(latitude=slice(closest_lat_n_coor, closest_lat_s_coor),
                                    longitude=slice(closest_lon_e_coor, closest_lon_w_coor)).values

        # Calculate statistics for the sliced data
        step_stats = calculate_statistics(temp_ds)
        for key in stats:
            stats[key].append(step_stats[key])

In [20]:
# new tests need to be done

import xarray as xr
import numpy as np
import pandas as pd
import sys
from datetime import datetime
import os

dataset_path = '/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/DATASETS/datasets_3h'
track_path = '/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/ALL_TRACKS/tracks_3h'

# Works for all years
# Define a function to open datasets and concatenate them
def open_and_concatenate(year, variable, months, way, level=0):
    datasets = []
    for month in months:
        dataset = xr.open_dataset(f'{way}{variable}/ERA5_{year}-{month}_{variable}.nc')
        if variable == 'geopotential' and level != 0:
            dataset = dataset.sel(level=level)
        
        # Create a date range with 3-hour intervals starting from midnight
        start = pd.Timestamp(f"{year}-{month}-01 00:00:00")
        if month == 12:
            end = pd.date_range(start=f"{year}-{month}-01", end=f"{str(int(year)+1)}-01-01", freq='M')[0] + pd.Timedelta(hours=21)
        else:
            end = pd.date_range(start=f"{year}-{month}-01", end=f"{year}-{month+1}-01", freq='M')[0] + pd.Timedelta(hours=21)
        date_range = pd.date_range(start, end, freq='3H')

        # Select the data at the specific timesteps
        dataset = dataset.sel(time=date_range)
        
        datasets.append(dataset)
        dataset.close()

    return xr.concat(datasets, dim='time')

# Define a function to calculate statistics
def calculate_statistics(data_array):
    return {
        'mean': np.mean(data_array),
        'min': np.min(data_array),
        'max': np.max(data_array),
        'std': np.std(data_array),
    }

# Function to log processing details
def log_processing(variable, year, level, storm_number):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_message = f'Processed variable: {variable}, Year: {year}, Level: {level}, Timestamp: {timestamp}, Storm number:{storm_number}'
    with open(f'{dataset_path}/processing_log_3h.txt', 'a') as log_file:
        log_file.write(log_message + '\n')

# Function to check if all CSV files exist
def all_csv_files_exist(variable, year, level):
    directory = f'{dataset_path}/{variable}'
#    if not os.path.exists(directory):
#       return False

    for storm_dir in os.listdir(directory):
        storm_path = os.path.join(directory, storm_dir)
        if os.path.isdir(storm_path):
            for stat in ['mean', 'min', 'max', 'std']:
                file_path = os.path.join(storm_path, f'{stat}_{storm_dir.split("_")[1]}_{level}.csv')
                if not os.path.exists(file_path):
                    return False
    return True

# Main function to process data
def process_data(variable, year, level=0):
    year = int(year)
    year_next = year + 1
    month_act = [10, 11, 12]
    month_next = [1, 2, 3]
    if variable == 'geopotential':
        way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5_hourly_PL/'
    else:
        way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'

    # Open and concatenate datasets
    if year == 1990:
        dataset_act = open_and_concatenate(str(year), variable, month_next, way, level)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')
        dataset = dataset.chunk({'time': 10})
    elif year == 2021:
        dataset = open_and_concatenate(str(year), variable, month_next, way, level)
    else:
        dataset_act = open_and_concatenate(str(year), variable, month_act, way, level)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')
        dataset = dataset.chunk({'time': 10})

    # Determine the specific variable to extract
    specific_var = next(var for var in dataset.variables if var not in ['longitude', 'latitude', 'time', 'level'])

    # Import all tracks and convert dates
    dates = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/storms_start_end_steps.csv', parse_dates=['start_date', 'end_date'])
    dates['year'] = dates['start_date'].dt.year

    # Find the indices for storms within the specified timeframe
    if year == 1990:
        index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
        index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]
    elif year == 2021:
        index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
        index_end_march = dates[(dates['end_date'].dt.year == 2021)].index[0]
    else:
    # Chercher start_october dans year, sinon chercher dès janvier de year_next
        index_start_october = dates[((dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)) | ((dates['start_date'].dt.year == year_next) & (dates['start_date'].dt.month >= 1))].index[0]
        index_end_march_first = dates[((dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next))].index
        #print(index_start_october, index_end_march_first, '3rd condition start_october + index_end_march_first')
        if len(index_end_march_first) > 0:
            index_end_march = index_end_march_first[-1]
            #print(index_end_march, 'index_end_march 1st condition of 2nd condition')
        else:
            # Si year_next ne renvoie rien, chercher la dernière instance de tempête dans year
            index_end_march = dates[((dates['end_date'].dt.year == year) & (dates['end_date'].dt.month <= 12))].index[-1]
            #print(index_end_march, 'index_end_march 2nd condition of 2nd condition')
    # Process each storm
    for i in range(index_start_october, index_end_march + 1):
        track = pd.read_csv(f'{track_path}/storm_{i+1}.csv')
        start_date = dates.at[i, 'start_date']
        end_date = dates.at[i, 'end_date']
        storm_data = dataset[specific_var].sel(time=slice(start_date, end_date))

        # Initialize lists to store statistics
        stats = {'mean': [], 'min': [], 'max': [], 'std': []}
        #, 'skewness': [], 'kurtosis': []
        
        # Process each time step
        for t_index in range(0, len(storm_data.time)):#, time_step in enumerate(storm_data.time):
            #data_slice = storm_data.sel(time=time_step).values

            # Extract coordinates for the current time step
            lon_e_temp = track.loc[t_index, 'lon_east']
            lon_w_temp = track.loc[t_index, 'lon_west']
            lat_s_temp = track.loc[t_index, 'lat_south']
            lat_n_temp = track.loc[t_index, 'lat_north']
            
            lon_test = np.asanyarray(storm_data.longitude[:])
            lat_test = np.asanyarray(storm_data.latitude[:])

            closest_lon_w = np.abs(lon_test - lon_w_temp).argmin()
            closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
            closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
            closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

            closest_lon_w_coor = lon_test[closest_lon_w]
            closest_lon_e_coor = lon_test[closest_lon_e]
            closest_lat_s_coor = lat_test[closest_lat_s]
            closest_lat_n_coor = lat_test[closest_lat_n]

            # Use .roll to handle the 0°/360° boundary
            if closest_lon_w_coor < 100 and closest_lon_e_coor > 100:
                roll_shift = {'longitude': int(round(closest_lon_w_coor, 0)), 'longitude': int(round(closest_lon_e_coor, 0))}
                storm_data_rolled = storm_data.roll(roll_shift, roll_coords=True)
            else:
                storm_data_rolled = storm_data

            # Slice the dataset based on the rolled longitudes and latitudes
            temp_ds_time = storm_data_rolled.isel(time=t_index)#[specific_var].isel(time=t_index)
            temp_ds = temp_ds_time.sel(latitude=slice(closest_lat_n_coor, closest_lat_s_coor),
                                       longitude=slice(closest_lon_e_coor, closest_lon_w_coor)).values

            # Calculate statistics for the sliced data
            step_stats = calculate_statistics(temp_ds)
            for key in stats:
                stats[key].append(step_stats[key])
                
        # Save statistics to CSV files
        for key in stats:
            directory = f'{dataset_path}/{variable}/storm_{i+1}'
            if not os.path.exists(directory):
                os.makedirs(directory)
            pd.DataFrame(stats[key]).to_csv(f'{directory}/{key}_{i+1}_{level}.csv')

        # Log the processing details
        log_processing(variable, year, level, i+1)

'''if __name__ == '__main__':
    variable = sys.argv[1]
    year = sys.argv[2]
    level = int(sys.argv[3])
    process_data(variable, year, level)'''

#if __name__ == '__main__':
variable = '2m_temperature'
year = 2013
level = 0

process_data(variable, year, level)

'''if not all_csv_files_exist(variable, year, level):
    process_data(variable, year, level)
else:
    print(f'All CSV files for variable: {variable}, year: {year}, and level: {level} already exist. Skipping processing.')'''

"if not all_csv_files_exist(variable, year, level):\n    process_data(variable, year, level)\nelse:\n    print(f'All CSV files for variable: {variable}, year: {year}, and level: {level} already exist. Skipping processing.')"

In [22]:
year = int(year)
year_next = year + 1
month_act = [10, 11, 12]
month_next = [1, 2, 3]
if variable == 'geopotential':
    way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5_hourly_PL/'
else:
    way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'

# Open and concatenate datasets
if year == 1990:
    dataset_act = open_and_concatenate(str(year), variable, month_next, way, level)
    dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
    dataset = xr.concat([dataset_act, dataset_next], dim='time')
    dataset = dataset.chunk({'time': 10})
elif year == 2021:
    dataset = open_and_concatenate(str(year), variable, month_next, way, level)
else:
    dataset_act = open_and_concatenate(str(year), variable, month_act, way, level)
    dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
    dataset = xr.concat([dataset_act, dataset_next], dim='time')
    dataset = dataset.chunk({'time': 10})

# Determine the specific variable to extract
specific_var = next(var for var in dataset.variables if var not in ['longitude', 'latitude', 'time', 'level'])

# Import all tracks and convert dates
dates = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/updated_storms_with_steps_and_index_ordered.csv', 
                    parse_dates=['start_date', 'end_date'])
dates['year'] = dates['start_date'].dt.year
dates.drop(columns=['Unnamed: 0'], inplace=True)

# Find the indices for storms within the specified timeframe
if year == 1990:
    index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]
elif year == 2021:
    index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.year == 2021)].index[0]
else:
# Chercher start_october dans year, sinon chercher dès janvier de year_next
    index_start_october = dates[((dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)) | ((dates['start_date'].dt.year == year_next) & (dates['start_date'].dt.month >= 1))].index[0]
    index_end_march_first = dates[((dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next))].index
    #print(index_start_october, index_end_march_first, '3rd condition start_october + index_end_march_first')
    if len(index_end_march_first) > 0:
        index_end_march = index_end_march_first[-1]
        #print(index_end_march, 'index_end_march 1st condition of 2nd condition')
    else:
        # Si year_next ne renvoie rien, chercher la dernière instance de tempête dans year
        index_end_march = dates[((dates['end_date'].dt.year == year) & (dates['end_date'].dt.month <= 12))].index[-1]
        #print(index_end_march, 'index_end_march 2nd condition of 2nd condition')
# Process each storm
for i in range(index_start_october, index_end_march + 1):
    storm_number = dates.at[i, 'storm_index']
    track = pd.read_csv(f'{track_path}/storm_{storm_number}.csv')
    start_date = dates.at[i, 'start_date']
    end_date = dates.at[i, 'end_date']
    storm_data = dataset[specific_var].sel(time=slice(start_date, end_date))

    # Initialize lists to store statistics
    stats = {'mean': [], 'min': [], 'max': [], 'std': []}
    #, 'skewness': [], 'kurtosis': []
    
    # Process each time step
    for t_index in range(0, len(storm_data.time)):#, time_step in enumerate(storm_data.time):
        #data_slice = storm_data.sel(time=time_step).values

        # Extract coordinates for the current time step
        lon_e_temp = track.loc[t_index, 'lon_east']
        lon_w_temp = track.loc[t_index, 'lon_west']
        lat_s_temp = track.loc[t_index, 'lat_south']
        lat_n_temp = track.loc[t_index, 'lat_north']
        
        lon_test = np.asanyarray(storm_data.longitude[:])
        lat_test = np.asanyarray(storm_data.latitude[:])

        closest_lon_w = np.abs(lon_test - lon_w_temp).argmin()
        closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
        closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
        closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

        closest_lon_w_coor = lon_test[closest_lon_w]
        closest_lon_e_coor = lon_test[closest_lon_e]
        closest_lat_s_coor = lat_test[closest_lat_s]
        closest_lat_n_coor = lat_test[closest_lat_n]

        # Use .roll to handle the 0°/360° boundary
        if closest_lon_w_coor < 100 and closest_lon_e_coor > 100:
            roll_shift = {'longitude': int(round(closest_lon_w_coor, 0)), 'longitude': int(round(closest_lon_e_coor, 0))}
            storm_data_rolled = storm_data.roll(roll_shift, roll_coords=True)
        else:
            storm_data_rolled = storm_data

        # Slice the dataset based on the rolled longitudes and latitudes
        temp_ds_time = storm_data_rolled.isel(time=t_index)#[specific_var].isel(time=t_index)
        temp_ds = temp_ds_time.sel(latitude=slice(closest_lat_n_coor, closest_lat_s_coor),
                                    longitude=slice(closest_lon_e_coor, closest_lon_w_coor)).values

        # Calculate statistics for the sliced data
        step_stats = calculate_statistics(temp_ds)
        for key in stats:
            stats[key].append(step_stats[key])


    # Save statistics to CSV files
    for key in stats:
        directory = f'{dataset_path}/{variable}/storm_{storm_number}'
        if not os.path.exists(directory):
            os.makedirs(directory)
        pd.DataFrame(stats[key]).to_csv(f'{directory}/{key}_{storm_number}_{level}.csv')

    # Log the processing details
    log_processing(variable, year, level, storm_number)

AttributeError: Failed to decode variable '2t': module 'numpy' has no attribute 'matrix'

In [16]:
# Import all tracks and convert dates
storm_data

<xarray.DataArray '2t' (time: 516, latitude: 721, longitude: 1440)>
dask.array<getitem, shape=(516, 721, 1440), dtype=float32, chunksize=(10, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 2013-10-12T06:00:00 ... 2013-12-15T15:00:00
Attributes: (12/14)
    long_name:                     2 metre temperature
    short_name:                    2t
    units:                         K
    original_format:               WMO GRIB 1 with ECMWF local table
    ecmwf_local_table:             128
    ecmwf_parameter:               167
    ...                            ...
    grid_specification:            0.25 degree x 0.25 degree from 90N to 90S ...
    rda_dataset:                   ds633.0
    rda_dataset_url:               https:/rda.ucar.edu/datasets/ds633.0/
    rda_dataset_doi:               DOI: 10.5065/BH6N-5N20
    rda_dataset_group:             ERA5 atmospheric surface analysis [netCDF4]
    number_of_significant_digits:  7

In [1]:
import pandas as pd

'''# Import all tracks and convert dates
dates = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/storms_start_end.csv', parse_dates=['start_date', 'end_date'])
dates['year'] = dates['start_date'].dt.year

# Find the indices for storms within the specified timeframe
if year == 1990:
    index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]
elif year == 2021:
    index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.year == 2021)].index[0]
else:'''
# Chercher start_october dans year, sinon chercher dès janvier de year_next
    index_start_october = dates[((dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)) | ((dates['start_date'].dt.year == year_next) & (dates['start_date'].dt.month >= 1))].index[0]
    index_end_march_first = dates[((dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next))].index
    #print(index_start_october, index_end_march_first, '3rd condition start_october + index_end_march_first')
    if len(index_end_march_first) > 0:
        index_end_march = index_end_march_first[-1]
        #print(index_end_march, 'index_end_march 1st condition of 2nd condition')
    else:
        # Si year_next ne renvoie rien, chercher la dernière instance de tempête dans year
        index_end_march = dates[((dates['end_date'].dt.year == year) & (dates['end_date'].dt.month <= 12))].index[-1]
        #print(index_end_march, 'index_end_march 2nd condition of 2nd condition')
#index_start_october = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/ALL_TRACKS/just_in_case/storm_{i+1}.csv')
# Process each storm
for i in range(1,97):#index_start_october, index_end_march + 1):
    dates = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/ALL_TRACKS/just_in_case/storm_{i+1}.csv')
    dates = dates['Date']
    year = int(dates[:4])
    month = int(dates[4:6])
    day = int(dates[6:8])
    hour = int(dates[8:10])

    # Create a datetime object
    dt = datetime(year, month, day, hour)

    # Format the datetime object into the desired string format
    formatted_date = dt.strftime("%Y-%m-%d %H:00:00")
    track = pd.read_csv(f'{track_path}/storm_{i+1}.csv')
    start_date = dates.at[i, 'start_date']
    end_date = dates.at[i, 'end_date']
    storm_data = dataset[specific_var].sel(time=slice(start_date, end_date))

    # Initialize lists to store statistics
    stats = {'mean': [], 'min': [], 'max': [], 'std': []}
    #, 'skewness': [], 'kurtosis': []
    
    # Process each time step
    for t_index in range(0, len(storm_data.time)):#, time_step in enumerate(storm_data.time):
        #data_slice = storm_data.sel(time=time_step).values

        # Extract coordinates for the current time step
        lon_e_temp = track.loc[t_index, 'lon_east']
        lon_w_temp = track.loc[t_index, 'lon_west']
        lat_s_temp = track.loc[t_index, 'lat_south']
        lat_n_temp = track.loc[t_index, 'lat_north']
        
        lon_test = np.asanyarray(storm_data.longitude[:])
        lat_test = np.asanyarray(storm_data.latitude[:])

        closest_lon_w = np.abs(lon_test - lon_w_temp).argmin()
        closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
        closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
        closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

        closest_lon_w_coor = lon_test[closest_lon_w]
        closest_lon_e_coor = lon_test[closest_lon_e]
        closest_lat_s_coor = lat_test[closest_lat_s]
        closest_lat_n_coor = lat_test[closest_lat_n]

        # Use .roll to handle the 0°/360° boundary
        if closest_lon_w_coor < 100 and closest_lon_e_coor > 100:
            roll_shift = {'longitude': int(round(closest_lon_w_coor, 0)), 'longitude': int(round(closest_lon_e_coor, 0))}
            storm_data_rolled = storm_data.roll(roll_shift, roll_coords=True)
        else:
            storm_data_rolled = storm_data

        # Slice the dataset based on the rolled longitudes and latitudes
        temp_ds_time = storm_data_rolled.isel(time=t_index)#[specific_var].isel(time=t_index)
        temp_ds = temp_ds_time.sel(latitude=slice(closest_lat_n_coor, closest_lat_s_coor),
                                    longitude=slice(closest_lon_e_coor, closest_lon_w_coor)).values

        # Calculate statistics for the sliced data
        step_stats = calculate_statistics(temp_ds)
        for key in stats:
            stats[key].append(step_stats[key])

NameError: name 'year' is not defined

In [3]:
import xarray as xr
import numpy as np
import pandas as pd
import sys
from datetime import datetime
import os

dataset_path = '/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/DATASETS/datasets_3h'
track_path = '/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/ALL_TRACKS/tracks_3h'

# Works for all years
# Define a function to open datasets and concatenate them
def open_and_concatenate(year, variable, months, way, level=0):
    datasets = []
    for month in months:
        dataset = xr.open_dataset(f'{way}{variable}/ERA5_{year}-{month}_{variable}.nc')
        if variable == 'geopotential' and level != 0:
            dataset = dataset.sel(level=level)
        
        # Create a date range with 3-hour intervals starting from midnight
        start = pd.Timestamp(f"{year}-{month}-01 00:00:00")
        if month == 12:
            end = pd.date_range(start=f"{year}-{month}-01", end=f"{str(int(year)+1)}-01-01", freq='M')[0] + pd.Timedelta(hours=21)
        else:
            end = pd.date_range(start=f"{year}-{month}-01", end=f"{year}-{month+1}-01", freq='M')[0] + pd.Timedelta(hours=21)
        date_range = pd.date_range(start, end, freq='3H')

        # Select the data at the specific timesteps
        dataset = dataset.sel(time=date_range)
        
        datasets.append(dataset)
        dataset.close()

    return xr.concat(datasets, dim='time')

# Define a function to calculate statistics
def calculate_statistics(data_array):
    return {
        'mean': np.mean(data_array),
        'min': np.min(data_array),
        'max': np.max(data_array),
        'std': np.std(data_array),
    }

# Function to log processing details
def log_processing(variable, year, level, storm_number):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_message = f'Processed variable: {variable}, Year: {year}, Level: {level}, Timestamp: {timestamp}, Storm number:{storm_number}'
    with open(f'{dataset_path}/processing_log_3h.txt', 'a') as log_file:
        log_file.write(log_message + '\n')

# Function to check if all CSV files exist
def all_csv_files_exist(variable, year, level):
    directory = f'{dataset_path}/{variable}'
    if not os.path.exists(directory):
        return False

    for storm_dir in os.listdir(directory):
        storm_path = os.path.join(directory, storm_dir)
        if os.path.isdir(storm_path):
            for stat in ['mean', 'min', 'max', 'std']:
                file_path = os.path.join(storm_path, f'{stat}_{storm_dir.split("_")[1]}_{level}.csv')
                if not os.path.exists(file_path):
                    return False
    return True

# Main function to process data
def process_data(variable, year, level=0):
    year = int(year)
    year_next = year + 1
    month_act = [10, 11, 12]
    month_next = [1, 2, 3]
    if variable == 'geopotential':
        way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5_hourly_PL/'
    else:
        way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'

    # Open and concatenate datasets
    if year == 1990:
        dataset_act = open_and_concatenate(str(year), variable, month_next, way, level)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')
        dataset = dataset.chunk({'time': 10})
    elif year == 2021:
        dataset = open_and_concatenate(str(year), variable, month_next, way, level)
    else:
        dataset_act = open_and_concatenate(str(year), variable, month_act, way, level)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')
        dataset = dataset.chunk({'time': 10})

    # Determine the specific variable to extract
    specific_var = next(var for var in dataset.variables if var not in ['longitude', 'latitude', 'time', 'level'])

    # Import all tracks and convert dates
    dates = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/updated_storms_with_steps_and_index_ordered.csv', 
                parse_dates=['start_date', 'end_date'])
    dates['year'] = dates['start_date'].dt.year
    dates.drop(columns=['Unnamed: 0'], inplace=True)

    # Find the indices for storms within the specified timeframe
    if year == 1990:
        index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
        index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]
    elif year == 2021:
        index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
        index_end_march = dates[(dates['end_date'].dt.year == 2021)].index[0]
    else:
    # Chercher start_october dans year, sinon chercher dès janvier de year_next
        index_start_october = dates[((dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)) | ((dates['start_date'].dt.year == year_next) & (dates['start_date'].dt.month >= 1))].index[0]
        index_end_march_first = dates[((dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next))].index
        #print(index_start_october, index_end_march_first, '3rd condition start_october + index_end_march_first')
        if len(index_end_march_first) > 0:
            index_end_march = index_end_march_first[-1]
            #print(index_end_march, 'index_end_march 1st condition of 2nd condition')
        else:
            # Si year_next ne renvoie rien, chercher la dernière instance de tempête dans year
            index_end_march = dates[((dates['end_date'].dt.year == year) & (dates['end_date'].dt.month <= 12))].index[-1]
            #print(index_end_march, 'index_end_march 2nd condition of 2nd condition')
    # Process each storm
    for i in range(index_start_october, index_end_march + 1):
        storm_number = dates.at[i, 'storm_index']
        track = pd.read_csv(f'{track_path}/storm_{storm_number}.csv')
        start_date = dates.at[i, 'start_date']
        end_date = dates.at[i, 'end_date']
        storm_data = dataset[specific_var].sel(time=slice(start_date, end_date))

        # Initialize lists to store statistics
        stats = {'mean': [], 'min': [], 'max': [], 'std': []}
        #, 'skewness': [], 'kurtosis': []
        
        # Process each time step
        for t_index in range(0, len(storm_data.time)):#, time_step in enumerate(storm_data.time):
            #data_slice = storm_data.sel(time=time_step).values

            # Extract coordinates for the current time step

            lon_e_temp = track.loc[t_index, 'lon_east']
            lon_w_temp = track.loc[t_index, 'lon_west']
            lat_s_temp = track.loc[t_index, 'lat_south']
            lat_n_temp = track.loc[t_index, 'lat_north']
            
            lon_test = np.asanyarray(storm_data.longitude[:])
            lat_test = np.asanyarray(storm_data.latitude[:])

            closest_lon_w = np.abs(lon_test - lon_w_temp).argmin()
            closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
            closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
            closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

            closest_lon_w_coor = lon_test[closest_lon_w]
            closest_lon_e_coor = lon_test[closest_lon_e]
            closest_lat_s_coor = lat_test[closest_lat_s]
            closest_lat_n_coor = lat_test[closest_lat_n]

            # Use .roll to handle the 0°/360° boundary
            if closest_lon_w_coor < 100 and closest_lon_e_coor > 100:
                roll_shift = {'longitude': int(round(closest_lon_w_coor, 0)), 'longitude': int(round(closest_lon_e_coor, 0))}
                storm_data_rolled = storm_data.roll(roll_shift, roll_coords=True)
            else:
                storm_data_rolled = storm_data

            # Slice the dataset based on the rolled longitudes and latitudes
            temp_ds_time = storm_data_rolled.isel(time=t_index)#[specific_var].isel(time=t_index)
            temp_ds = temp_ds_time.sel(latitude=slice(closest_lat_n_coor, closest_lat_s_coor),
                                       longitude=slice(closest_lon_e_coor, closest_lon_w_coor)).values

            # Calculate statistics for the sliced data
            step_stats = calculate_statistics(temp_ds)
            for key in stats:
                stats[key].append(step_stats[key])


        # Save statistics to CSV files
        for key in stats:
            directory = f'{dataset_path}/{variable}/storm_{storm_number}'
            if not os.path.exists(directory):
                os.makedirs(directory)
            pd.DataFrame(stats[key]).to_csv(f'{directory}/{key}_{storm_number}_{level}.csv')

        # Log the processing details
        log_processing(variable, year, level, storm_number)


variable = '2m_temperature'
year = 2013
level = 0
process_data(variable, year, level)

'''if __name__ == '__main__':
    variable = sys.argv[1]
    year = sys.argv[2]
    level = int(sys.argv[3])

    if not all_csv_files_exist(variable, year, level):
        process_data(variable, year, level)
    else:
        print(f'All CSV files for variable: {variable}, year: {year}, and level: {level} already exist. Skipping processing.')'''

"if __name__ == '__main__':\n    variable = sys.argv[1]\n    year = sys.argv[2]\n    level = int(sys.argv[3])\n\n    if not all_csv_files_exist(variable, year, level):\n        process_data(variable, year, level)\n    else:\n        print(f'All CSV files for variable: {variable}, year: {year}, and level: {level} already exist. Skipping processing.')"